In [1]:
from pathlib import Path

import eelbrain
import mne
from scipy.io import loadmat
import numpy as np



# Data locations
DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'
DATA_PREPROC = DATA_ROOT / "Data Preprocessing"



In [9]:

for file in DATA_PREPROC.glob("*.mat"):

    SUBJECT = file.stem.split("_")[0]

    dst_dir = DATA_ROOT / 'envelopes' / SUBJECT
    dst_dir.mkdir(exist_ok=True, parents=True)
    attended_dst = dst_dir / f'attended_{file.stem}_raw.fif'
    unattended_dst = dst_dir / f'unattended_{file.stem}_raw.fif'
    if attended_dst.exists() and unattended_dst.exists():
        print(f"Files for {SUBJECT} already exist, skipping.")
        continue


    print("Saving ~", SUBJECT, "...")

    mat = loadmat(file, squeeze_me=True, struct_as_record=False)
    data = mat['data']

    # --- collect all trials ---
    att_envelope_trials = []
    unatt_envelope_trials = []

    for att_envelope, unatt_envelope in zip(data.wavA, data.wavB):  # 60 trials
        att_envelope_trials.append(att_envelope)
        unatt_envelope_trials.append(unatt_envelope)

    # --- concatenate trials ---
    att_envelope_concat = np.concatenate(att_envelope_trials)
    unatt_envelope_concat = np.concatenate(unatt_envelope_trials)

    # Create time axis
    time_axis = eelbrain.UTS(0, 1/64, len(att_envelope_concat))  # assuming 64 Hz

    # Save the attended and unattended envelopes as NDVars
    att_ndvar = eelbrain.NDVar(att_envelope_concat, dims=(time_axis,), name='attended')
    eelbrain.save.pickle(att_ndvar, dst_dir / f'attended_envelope.pickle')

    unatt_ndvar = eelbrain.NDVar(unatt_envelope_concat, dims=(time_axis,), name='unattended')
    eelbrain.save.pickle(unatt_ndvar, dst_dir / f'unattended_envelope.pickle')


Saving ~ S5 ...
Saving ~ S15 ...
Saving ~ S11 ...
Saving ~ S1 ...
Saving ~ S4 ...
Saving ~ S14 ...
Saving ~ S10 ...
Saving ~ S17 ...
Saving ~ S7 ...
Saving ~ S18 ...
Saving ~ S3 ...
Saving ~ S13 ...
Saving ~ S8 ...
Saving ~ S16 ...
Saving ~ S6 ...
Saving ~ S2 ...
Saving ~ S9 ...
Saving ~ S12 ...
